In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [3]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8042
WEBHOOK_BASE = ""
ADMIN_URL = "http://mediator1-agent:8041"

In [4]:
# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, mediation=True)
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)

In [5]:
# Create Invitation
invite = await agent_controller.connections.create_invitation(multi_use="true")
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)

Connection ID eeab0d84-a6ac-42ed-8d60-5813734de0f7
Invitation
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '90a677dc-c355-4652-9ccd-623716045f56', 'recipientKeys': ['9SqB6GmysNaHDqfLoRSjP49wDsji96eX9DHYpJQu977M'], 'serviceEndpoint': 'https://9ec92737999e.ngrok.io', 'label': 'MEDIATOR1'}
Connection Handler Called
Connection eeab0d84-a6ac-42ed-8d60-5813734de0f7 in State invitation
Connection Handler Called
Connection eeab0d84-a6ac-42ed-8d60-5813734de0f7 in State invitation


In [7]:
response = await agent_controller.mediation.get_mediation_records()

for record in response:
    print("Mediation Record")
    print("connection_id", record["connection_id"])
    print("State", record["state"])